# Import Necessary Packages

In [1]:
import numpy as np
import os
import io
import PIL.Image as Image
from array import array
from imutils import perspective
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import cv2
from scipy.spatial import distance
from IPython.display import display, Markdown, clear_output
import tensorflow as tf
from tensorflow.keras import layers
from scipy.interpolate import splprep, splev
from tensorflow import keras
keras.backend.set_image_data_format('channels_last')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import numpy
from pathlib import Path
from scipy.spatial import distance
import math
import sys
from scipy.ndimage import gaussian_filter
from random import choice
import shutil
import time
import panel as pn
from fpdf import FPDF
pn.extension()
from tensorflow.keras.models import Model, Sequential, load_model
import random
import pandas as pd
from io import BytesIO
from os.path import exists
#pn.extension(sizing_mode="stretch_width")
random.seed(10)

# Functions for Pre and Post Processing

In [2]:
def find_start_position(image,point):
    for n in range(point[0], image.shape[1]):
        max_value = 0.
        max_value_index= 0
        for m in range(point[1],image.shape[0]):
             if image[m,n] > max_value:
                max_value = image[m,n]
                max_value_index= m
        if(max_value>0):
            break
    start = (n,max_value_index)
    return start

def find_continuous_line(image, point):
    points_list=[]
    points_list.append(point)
    
    while(1):
        
        current_x = points_list[-1][0]
        current_y = points_list[-1][1]
        
        if current_x+1>image.shape[1]-1:
            break
        if current_y-1<0:
            break
        if current_y+1>image.shape[0]-1:
            break
        
        value = []
        value = (image[current_y+1, current_x+1],
                     image[current_y, current_x+1],
                     image[current_y-1, current_x+1])
        
        maximum = max(value)
        if maximum==0:
            break
        else:
            if (image[current_y+1, current_x+1]==maximum):
                points_list.append((current_x+1, current_y+1))
            elif (image[current_y, current_x+1]==maximum):
                points_list.append((current_x+1, current_y))
            else:
                points_list.append((current_x+1, current_y-1))
                
    return points_list[1:]

def find_continuous_line_for_tooth_bone_area(image, point):
    points_list=[]
    points_list.append(point)
    
    
    while(1):
        current_x = points_list[-1][0]
        current_y = points_list[-1][1]
        
        if current_x+1>image.shape[1]-1:
            break
            
        value = image[:, current_x+1]
        maximum = max(value)
        if maximum==0:
            break
        else:
            current_y = np.nonzero(value)
            points_list.append((current_x+1, current_y[0][0]))
    return points_list[1:]

def findIntersection(x1,y1,x2,y2,x3,y3,x4,y4):
        px= ( (x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) ) 
        py= ( (x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) )
        return (int(px), int(py))
    
def show_mask(mask):
    visible_mask = np.zeros((mask.shape[0],mask.shape[1]), dtype = np.uint8)
    for i in range (mask.shape[0]):
        for j in range(mask.shape[1]):
            if(mask[i,j]==1):
                visible_mask[i,j] = 255
    return visible_mask

# Load Models

In [3]:
pretrianed_bone_area_model=None
pretrianed_tooth_model=None
pretrianed_cej_line_model = None

pretrianed_bone_area_model = keras.models.load_model('./model_folder/model_for_bone_area_resnet_34.h5')
pretrianed_tooth_model = keras.models.load_model('./model_folder/model_for_tooth_resnet_34_512.h5')
pretrianed_cej_line_model = keras.models.load_model('./model_folder/model_for_cej.h5')


# Bone Area Predcition

In [4]:
def bone_area_prediction(image_file_name):
    
    image = image_file_name
    cv2.imwrite('test.png',image)
    
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    IMG_CHANNELS = 3
    X_test = np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    
    img = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    X_test[0] = img
    prediction_for_test = pretrianed_bone_area_model.predict(X_test, verbose=1)
    temp = prediction_for_test[0,:,:,0]
    cv2.imwrite('test.png',temp)
    img_y = cv2.imread('test.png',0)
    img_y = show_mask(img_y)
    img_y = cv2.resize(img_y,(image.shape[1],image.shape[0]))
    
    img = cv2.GaussianBlur(img_y,(5,5),1)
    ret,thresh = cv2.threshold(img,127,255,0)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    contoursSorted = sorted(contours, key=lambda x: cv2.contourArea(x), reverse = True)
    img_color = np.zeros((image.shape[0], image.shape[1], 3))
      
    smoothened = []
    
    for c in contoursSorted:
        if (cv2.contourArea(c)>4000):
            x,y = c.T
            x = x.tolist()[0]
            y = y.tolist()[0]
            tck, u = splprep([x,y], u=None, s=1.0, per=1)
            u_new = numpy.linspace(u.min(), u.max(), 70)
            x_new, y_new = splev(u_new, tck, der=0)
            res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new,y_new)]
            smoothened.append(numpy.asarray(res_array, dtype=numpy.int32))
            cv2.fillPoly(img_color,smoothened,color = (255,255,255))
    return img_color

# Tooth Predcition

In [5]:
def tooth_prediction(image_file_name):
    
    
    image = image_file_name
    
    IMG_HEIGHT = 512
    IMG_WIDTH = 512
    IMG_CHANNELS = 3
    X_test = np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    
    img = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    X_test[0] = img
    prediction_for_test = pretrianed_tooth_model.predict(X_test, verbose=1)
    temp = prediction_for_test[0,:,:,0]
    
    cv2.imwrite('test.png',temp)
    img_y = cv2.imread('test.png',0)
    
    img_y = show_mask(img_y)
    img_y = cv2.resize(img_y,(image.shape[1],image.shape[0]))
    
    
    img = cv2.GaussianBlur(img_y,(5,5),1)
    ret,thresh = cv2.threshold(img,127,255,0)

    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contoursSorted = sorted(contours, key=lambda x: cv2.contourArea(x), reverse = True)
    img_color = np.zeros((image.shape[0], image.shape[1], 3))

        
    smoothened = []
    
    for c in contoursSorted:
        if (cv2.contourArea(c)>5000):
            x,y = c.T
            x = x.tolist()[0]
            y = y.tolist()[0]
            tck, u = splprep([x,y], u=None, s=1.0, per=1)
            u_new = numpy.linspace(u.min(), u.max(), 70)
            x_new, y_new = splev(u_new, tck, der=0)
            res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new,y_new)]
            smoothened.append(numpy.asarray(res_array, dtype=numpy.int32))
            cv2.fillPoly(img_color,smoothened,color = (255,255,255))
    return img_color

# CEJ Line Prediction

In [6]:
def cej_line_prediction(image_file_name):
    
    
    image = image_file_name
    
    IMG_HEIGHT =128
    IMG_WIDTH =128
    IMG_CHANNELS = 3

    X_test = np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

    img = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    X_test[0] = img
    prediction_for_test = pretrianed_cej_line_model.predict(X_test, verbose=1)
    temp = prediction_for_test[0,:,:,0]
    cv2.imwrite('test.png',temp)
    img_y = cv2.imread('test.png',0)
    img_y = show_mask(img_y)
    img_y = cv2.resize(img_y,(image.shape[1],image.shape[0]))
    
    img = img_y
    
    img_binary = np.zeros((img.shape[0], img.shape[1]))
    for p in range(img.shape[0]):
        for q in range(img.shape[1]):
            if(img[p,q] > 128):
                img_binary[p,q] = 255
               
    kernel = np.ones((7,7),np.uint8)
    img_binary = gaussian_filter(img_binary, sigma=(7,7))
    img_binary = cv2.dilate(img_binary,kernel,iterations = 1)
    img_binary = cv2.erode(img_binary,kernel,iterations = 1)

    img_filter = np.zeros((img.shape[0], img.shape[1]))
    for m in range(img.shape[0]-2):
        for n in range(img.shape[1]-2):
            img_filter[m+1,n+1] = np.sum(img_binary[m:m+3,n:n+3])
            
    segments = []
    segments_debug = []
    initial = (0,0)
    while initial[0] < img_filter.shape[1]:
        start_line = find_start_position(img_filter,initial)
        all_points = []
        all_points.append(start_line)
        continuous_points = find_continuous_line(img_filter, start_line)
        all_points.extend(continuous_points)

        segments_debug.append(all_points)
        segments.extend(all_points)
        last_point = all_points[-1]
        initial = (last_point[0]+1, max(last_point[1]-50, 0))
    
    img_draw = np.zeros((img.shape[0], img.shape[1],3))
    for pp in range(len(segments)-2):
        cv2.line(img_draw,(segments[pp][0],segments[pp][1]),(segments[pp+1][0],segments[pp+1][1]),(255,255,255),1)
    
    return segments, img_draw

# Upper Lower jaw Determine

In [7]:
def upper_lower_jaw_determine(bone_img_file_name, cej_line_img_file_name):
    

    img_bone_area = bone_img_file_name
    img_cej = cej_line_img_file_name

    ret,thresh = cv2.threshold(img_bone_area,127,255,0)
    img_tooth_contours = np.zeros((img_bone_area.shape[0], img_bone_area.shape[1]),  dtype =numpy.uint8)
    ctrs, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(ctrs, key=lambda ctr: cv2.contourArea(ctr), reverse=True)

    ####moments calculation for each contour
    M = cv2.moments(contours[0])
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    corners = cv2.goodFeaturesToTrack(img_cej, 200, 0.05, 10)
    corner_list=[]

    for corner in corners:
        x, y = corner.ravel()
        corner_list.append((x,y))
    corner_list_soretd = sorted(corner_list, key=lambda point: point[0])
    v1 = (corner_list_soretd[-3][0]-corner_list_soretd[0][0], corner_list_soretd[-3][1]-corner_list_soretd[0][1])
    v2 = (corner_list_soretd[-3][0]-cx, corner_list_soretd[-3][1]-cy)
    xp = v1[0]*v2[1]-v1[1]*v2[0] 
    if (xp>=0):
        return 0
    else:
        return 1

# Main Function

In [10]:
input_directory =  'input_folder_choity/' 
output_directory =  'output_folder/'

file_input = os.listdir(input_directory)

for i in range(len(file_input)):
    
    cv2_image = cv2.imread(input_directory + file_input[i])
    temp_bone_area = bone_area_prediction(cv2_image)
    temp_tooth = tooth_prediction(cv2_image)
    cej_line_segments, temp_cej_line = cej_line_prediction(cv2_image)

    cv2.imwrite(output_directory + 'bone_area_'+ file_input[i], temp_bone_area)
    cv2.imwrite(output_directory + 'tooth_'+ file_input[i], temp_tooth)
    cv2.imwrite(output_directory + 'cej_line_'+ file_input[i], temp_cej_line)
    
    bone_area_2d = cv2.imread(output_directory + 'bone_area_'+ file_input[i], 0)
    tooth_2d = cv2.imread(output_directory + 'tooth_'+ file_input[i], 0)
    cej_line_2d = cv2.imread(output_directory + 'cej_line_'+ file_input[i], 0)

    upper_lower = upper_lower_jaw_determine(bone_area_2d, cej_line_2d)
    
    print(upper_lower)
    
    if (upper_lower ==1):
        print('Lower Jaw')
    elif (upper_lower ==0):
        print('Upper Jaw')
    else:
        print('Bitewing')
        

1/1 [==============================] - 0s 158ms/step
0
Upper Jaw
